[X]: E01: Tune the hyperparameters of the training to beat my best validation loss of 2.2

[X]: E02: I was not careful with the intialization of the network in this video. (1) What is the loss you'd get if the predicted probabilities at initialization were perfectly uniform? What loss do we achieve? (2) Can you tune the initialization to get a starting loss that is much more similar to (1)?

[X]: E03: Read the Bengio et al 2003 paper (link above), implement and try any idea from the paper. Did it work?

```
cd lectures/makemore
uv run tensorboard --logdir=lightning_logs/
uv run optuna-dashboard sqlite:///makemore_part2.db
```

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
from torch.utils.data import TensorDataset, DataLoader
from lightning.pytorch.callbacks import EarlyStopping
import optuna
from optuna.integration import PyTorchLightningPruningCallback

/Users/kevin/nn-zero-to-hero/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
context_length = 3

xs = []
ys = []

itos = {i + 1: chr(i + ord('a')) for i in range(26)} | {0: '.'}
stoi = {s:i for i,s in itos.items()}

for w in words:
    x = [0] * context_length
    w = w + '.'
    for c in w:
        xs.append(x)
        ix = stoi[c]
        ys.append(ix)
        x = x[1:] + [ix]

xs = torch.tensor(xs)
ys = torch.tensor(ys)

dataset = TensorDataset(xs, ys)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256)

In [12]:
class MLPBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim, activation, skip_connection):
        super().__init__()
        self.linear = nn.Linear(input_dim, hidden_dim)
        self.activation = activation()
        self.skip_connection = skip_connection
    
    def forward(self, x):
        out = self.linear(x)
        out = self.activation(out)
        if self.skip_connection:
            out = out + x
        return out
        

class MLP(L.LightningModule):
    def __init__(self, 
                 embedding_dim=8, 
                 hidden_dim=100, 
                 lr=0.001,
                 weight_decay=0.01,
                 activation="Tanh",
                 optimizer="AdamW",
                 num_layers=2,
                 skip_connection=True,
                 ):
        super().__init__()
        self.save_hyperparameters()

        activation = getattr(nn, self.hparams.activation)

        layers = [nn.Embedding(27, embedding_dim), nn.Flatten(-2)]
        for i in range(num_layers):
            if i == 0:
                input_dim = context_length * embedding_dim
                layer_skip_connection = False
            else:
                input_dim = hidden_dim
                layer_skip_connection = skip_connection
            layers.append(MLPBlock(input_dim, hidden_dim, activation, layer_skip_connection))
        layers.append(nn.Linear(hidden_dim, 27))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('val_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = getattr(torch.optim, self.hparams.optimizer)
        return optimizer(self.model.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)

In [ ]:
def objective(trial: optuna.trial.Trial):
    embedding_dim = trial.suggest_int("embedding_dim", 4, 27)
    hidden_dim = trial.suggest_int("hidden_dim", 10, 200, log=True)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e0, log=True)
    nonlinearity = trial.suggest_categorical("nonlinearity", ["Tanh", "ReLU"])
    optimizer = trial.suggest_categorical("optimizer", ["AdamW", "Adam", "SGD"])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    skip_connection = trial.suggest_categorical("skip_connection", [True, False])

    model = MLP(embedding_dim, hidden_dim, lr, weight_decay, nonlinearity, optimizer, num_layers, skip_connection)
    
    trainer = L.Trainer(
        accelerator="cpu",
        devices=1,
        max_epochs=100,
        callbacks=[
            EarlyStopping(monitor="val_loss", mode="min"),
            PyTorchLightningPruningCallback(trial, monitor="val_loss")
        ],
        enable_model_summary=False,
        enable_progress_bar=False,
        enable_checkpointing=False
    )
    trainer.fit(model, train_dataloader, val_dataloader)
    return trainer.callback_metrics["val_loss"]

study = optuna.create_study(
    storage="sqlite:///makemore_part2.db",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10, n_min_trials=3),
    study_name="makemore_part2",
    load_if_exists=True
)
study.optimize(objective, n_trials=100)

[I 2025-10-01 13:38:31,266] Using an existing study with name 'makemore_part2' instead of creating a new one.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/kevin/nn-zero-to-hero/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 4.1 K  | train
---------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


/Users/kevin/nn-zero-to-hero/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/kevin/nn-zero-to-hero/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 43: 100%|██████████| 713/713 [00:01<00:00, 542.59it/s, v_num=26]


[I 2025-10-01 13:39:31,353] Trial 28 finished with value: 2.2000160217285156 and parameters: {'embedding_dim': 25, 'hidden_dim': 33, 'lr': 0.0004665978183545832, 'weight_decay': 0.11228561032697912, 'nonlinearity': 'ReLU', 'optimizer': 'AdamW', 'num_layers': 1, 'skip_connection': False}. Best is trial 8 with value: 2.199681043624878.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 4.5 K  | train
---------------------------------------------
4.5 K     Trainable params
0         Non-trainable params
4.5 K     Total params
0.018     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Epoch 77: 100%|██████████| 713/713 [00:01<00:00, 522.78it/s, v_num=27]       


...

Epoch 12: 100%|██████████| 713/713 [00:02<00:00, 354.47it/s, v_num=66]      


[I 2025-10-01 14:08:09,642] Trial 68 finished with value: 2.1070501804351807 and parameters: {'embedding_dim': 26, 'hidden_dim': 193, 'lr': 0.0018092132814085924, 'weight_decay': 0.0013342154966771917, 'nonlinearity': 'Tanh', 'optimizer': 'AdamW', 'num_layers': 2, 'skip_connection': False}. Best is trial 56 with value: 2.0905239582061768.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 48.4 K | train
---------------------------------------------
48.4 K    Trainable params
0         Non-trainable params
48.4 K    Total params
0.194     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Epoch 16: 100%|██████████| 713/713 [00:01<00:00, 368.72it/s, v_num=67]      


[I 2025-10-01 14:08:43,402] Trial 69 finished with value: 2.1008408069610596 and parameters: {'embedding_dim': 23, 'hidden_dim': 175, 'lr': 0.0007334814191777619, 'weight_decay': 0.0004775110863698371, 'nonlinearity': 'Tanh', 'optimizer': 'AdamW', 'num_layers': 2, 'skip_connection': False}. Best is trial 56 with value: 2.0905239582061768.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 49.4 K | train
---------------------------------------------
49.4 K    Trainable params
0         Non-trainable params
49.4 K    Total params
0.198     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Epoch 12:  59%|█████▊    | 418/713 [00:01<00:01, 211.33it/s, v_num=68]      

In [ ]:
# init_model = MLP()
# init_loss = F.cross_entropy(init_model(xs), ys)
# print(init_loss)

tensor(3.3429, grad_fn=<NllLossBackward0>)


In [ ]:
# torch.log(torch.tensor(27))

tensor(3.2958)